# Poisson Equation in Parallel

NGSolve can be executed on a cluster using the MPI message passing interface.
You can download [poisson_mpi.py](poisson_mpi.py) and run it as

> mpirun -np 4 poisson_mpi.py 

The solution is saved such that we can visualize it with [drawsolution.py](drawsolution.py)
> netgen drawsolution.py

For proper parallel execution, Netgen/NGSolve must be configured with '-DUSE_MPI=ON'. Recent binaries for Linux and Mac are built with MPI support (?). If you are unsure if your Netgen/NGSolve supports MPI, look output like "Including MPI version 3.1" douring Netgen startup.

## MPI-parallel execution using ipyparallel

In the jupyter-tutorials we use ipyparallel. Please consult https://ipyparallel.readthedocs.io for installation.

I am starting the cluster on my notebook via
> ipcluster start --engines=MPI -n 4 --profile=mpi

In jupyter, we can then connect to the cluster via

In [ ]:
from ipyparallel import Client
c = Client(profile='mpi')

We use mpi4py https://mpi4py.readthedocs.io/ for issuing MPI calls from Python:

In [ ]:
%%px 
from mpi4py import MPI
comm = MPI.COMM_WORLD
print (comm.rank, comm.size)

The master process (rank==0) generates the mesh, and distributes it within the group of processes defined by the communicator. All other ranks receive a part of the mesh. The function mesh.GetNE(VOL) returns the local number of elements:

In [ ]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square

if comm.rank == 0:
    ngmesh = unit_square.GenerateMesh(maxh=0.02)
    ngmesh.Save("square.vol")
    ngmesh.Distribute(comm)
else:
    ngmesh = netgen.meshing.Mesh.Receive(comm)
mesh = Mesh(ngmesh)
print (mesh.GetNE(VOL))

We can define spaces, bilinear and linear forms, gridfunctions the same way as in sequential mode. But now, the degrees of freedom are distributed on the cluster following the distribution of the mesh. The finite element spaces defines how the dofs match together.

In [ ]:
%%px
fes = H1(mesh, order=3, dirichlet=".*")
u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
pre = Preconditioner(a, "local")
a.Assemble()

f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)

inv = CGSolver(a.mat, pre.mat)
gfu.vec.data = inv*f.vec

print (InnerProduct(gfu.vec, f.vec))

In [ ]:
%%px
# does not work, generate solution file using mpirun
gfu.Save("solution.sol", parallel=True)

To visualize the parallel solution, we setup the same space on the same mesh, and load the coefficient vector of the gridfunction. The flag 'parallel=True' ensures that the ordering of variables is the same as in the parallel run.

In [ ]:
# the files have to generated via an mpirun ...

from ngsolve import *
from ngsolve.webgui import Draw

mesh = Mesh("square.vol")
fes = H1(mesh, order=3)
gfu = GridFunction(fes)
gfu.Load("solution.sol", parallel=True)

Draw (gfu)